# Skillra PDA: HSE course project

**Вклады участников (placeholders):**
- Аналитик A — предобработка данных
- Аналитик B — фичи и визуализации
- Аналитик C — продуктовый слой и персоны


## 0. Импорт и загрузка данных

In [ ]:
from pathlib import Path
import sys
import importlib
import pandas as pd

def _find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'pyproject.toml').exists() or (candidate / 'src' / 'skillra_pda').exists():
            return candidate
    return start

_start_path = Path(__file__).resolve() if '__file__' in globals() else Path.cwd().resolve()
PROJECT_ROOT = _find_project_root(_start_path)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.skillra_pda import io, cleaning, features, eda, viz, personas, config

config.ensure_directories()
raw_path = config.RAW_DATA_FILE
raw_path

In [ ]:
df_raw = io.load_raw(raw_path)
df_raw.head(3)

## 1. Паспорт датасета и качество

In [ ]:

profile = cleaning.basic_profile(df_raw)
profile


In [ ]:

is_unique, dup_count = cleaning.check_unique_id(df_raw)
groups = cleaning.detect_column_groups(df_raw)
missing_top = eda.missing_share(df_raw)
{
    "unique_vacancy_id": is_unique,
    "duplicate_count": dup_count,
    "group_sizes": {k: len(v) for k, v in groups.items()},
    "missing_top10": missing_top.head(10),
}


### Выводы по качеству
- Проверили уникальность `vacancy_id` и доли пропусков.
- Сгруппировали наборы колонок по префиксам для дальнейших агрегатов.

## 2. Предобработка

In [ ]:

df = df_raw.copy()
df = cleaning.parse_dates(df)
df = cleaning.deduplicate(df)
df = cleaning.handle_missingness(df)
df = cleaning.salary_prepare(df)
df = cleaning.ensure_salary_gross_boolean(df)

if "salary_gross" in df.columns:
    assert str(df["salary_gross"].dtype) == "boolean", "salary_gross must be boolean"
    assert not df["salary_gross"].isin(["unknown", "Unknown", "UNKNOWN", ""]).any(), "salary_gross contains unknown markers"

df.shape


### Комментарии по предобработке
- Парсим даты, удаляем дубликаты по `vacancy_id`, обрабатываем пропуски.
- Зарплатные поля нормализуем и каппируем выбросы.
- `salary_gross` приводим к Nullable boolean с удалением строковых маркеров.

In [ ]:

clean_path = config.CLEAN_DATA_FILE
io.save_processed(df, clean_path)
clean_path


## 3. Новые признаки

In [ ]:

df_features = features.assemble_features(df.copy())
df_features = features.ensure_expected_feature_columns(df_features)

key_columns = [
    "published_weekday",
    "city_tier",
    "work_mode",
    "primary_role",
    "salary_bucket",
    "vacancy_age_days",
    "core_data_skills_count",
    "ml_stack_count",
    "tech_stack_size",
]
df_features[key_columns].head()


### Зачем эти признаки?
- `vacancy_age_days` — свежесть вакансии.
- `published_weekday` и `is_weekend_post` — сезонность публикаций.
- `city_tier` и `work_mode` — гео и формат работы.
- `primary_role` — приоритезация ролей.
- `core_data_skills_count` / `ml_stack_count` / `tech_stack_size` — размер технологического стека.
- Текстовые счётчики по описанию фиксируют насыщенность требований.

In [ ]:

feat_path = config.FEATURE_DATA_FILE
io.save_processed(df_features, feat_path)
feat_path


## 4. EDA

In [ ]:

snapshot = {
    "shape": df_features.shape,
    "salary_known_share": df_features["salary_known"].mean() if "salary_known" in df_features.columns else None,
    "median_salary_rub": df_features.get("salary_mid_rub_capped", pd.Series(dtype=float)).median(),
}
snapshot


In [ ]:
import importlib
eda = importlib.reload(eda)

salary_by_grade_city = eda.salary_summary_by_grade_and_city(df_features)
salary_by_role_mode = eda.salary_summary_by_role_and_work_mode(df_features)
remote_by_role = eda.remote_share_by_role(df_features)
junior_segment = eda.junior_friendly_share_by_segment(df_features)

{
    "salary_by_grade_city": salary_by_grade_city.head(),
    "salary_by_role_mode": salary_by_role_mode.head(),
    "remote_by_role": remote_by_role.head(),
    "junior_segment": junior_segment.head(),
}


### Инсайты EDA
- Зарплаты по грейдам и городским кластерам показывают различия по рынку.
- Remote share по ролям демонстрирует гибкость работодателей.
- Junior-friendly атрибуты агрегированы по ролям/грейдам для оценки доступности.

## 5. Визуализация

In [ ]:

import importlib
import src.skillra_pda.viz as viz
viz = importlib.reload(viz)

fig_salary_grade = viz.salary_by_grade_box(df_features)
fig_salary_role = viz.salary_by_role_box(df_features)
fig_workmode = viz.work_mode_share_by_city(df_features)

skill_cols = [c for c in df_features.columns if c.startswith("skill_") or c.startswith("has_")]
fig_skill_heatmap = viz.skill_heatmap(
    df_features,
    index_col="grade" if "grade" in df_features.columns else df_features.columns[0],
    skill_cols=skill_cols,
    title="Skill vs grade heatmap",
    filename="fig_skill_vs_grade_heatmap.png",
    top_n=20,
)

fig_salary_grade, fig_salary_role, fig_workmode, fig_skill_heatmap


### Выводы по графикам
- Boxplot по зарплатам позволяет быстро увидеть медианы и выбросы.
- Состав work mode по city tier показывает предпочитаемые форматы.
- Heatmap навыков × грейд визуализирует, какие навыки чаще требуются на каждом уровне.

## 6. Продуктовый слой: персоны

In [ ]:

student = personas.Persona(
    name="data_student",
    description="Магистрант по данным, целится в Junior DA/DS",
    current_skills=["skill_sql", "skill_excel", "has_python"],
    target_filter={"grade": "junior", "primary_role": ["data", "analyst", "ml"]},
)

switcher = personas.Persona(
    name="switcher_bi",
    description="Свитчер в продуктовую/BI-аналитику",
    current_skills=["skill_excel", "skill_powerbi"],
    target_filter={"grade": ["junior", "middle"], "primary_role": ["product", "analyst"]},
)

analyst = personas.Persona(
    name="mid_data_analyst",
    description="Middle аналитик, хочет усилить hard-стек",
    current_skills=["skill_sql", "skill_excel", "skill_powerbi"],
    target_filter={"grade": "middle", "primary_role": ["data", "analyst"]},
)

personas_list = [student, switcher, analyst]
results = {}
for persona in personas_list:
    gap = personas.skill_gap_for_persona(df_features, persona, top_k=15)
    results[persona.name] = {
        "gap_table": gap.head(10),
        "plot": personas.plot_persona_skill_gap(gap, persona),
    }

results


### Выводы по персонам
- Для студента приоритет — добрать продвинутый SQL и Python-стек.
- Свитчеру в BI важны BI-инструменты и продуктовые навыки.
- Middle-аналитику нужны ML/оркестрация для роста к DS/ML Ops.

## 7. Итоги
- Выполнены этапы 0–4 ТЗ: предобработка, фичи, EDA, визуализации.
- Сохранили чистый и фичевый датасеты, построили витрины графиков.
- Применили продуктовый слой через персоны и анализ skill gap.